# Plankton


In [28]:
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt

print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


Link naar waar ik code hieronder vandaan heb
https://drive.google.com/file/d/1hAaPzDMVEZ8X1tfRS2ieFEqi0R7Ww7uL/view

https://stackoverflow.com/questions/30230592/loading-all-images-using-imread-from-a-given-folder

In [29]:
test_data = []
folder_test = 'data/test'

for filename in os.listdir(folder_test):
    img = cv2.imread(os.path.join(folder_test, filename))
    test_data.append(img)


In [30]:
train_data = []
train_labels_not_categorical = []

folder_train = 'data/train'

for number, categories in enumerate(os.listdir(folder_train)):
    for filename in os.listdir(f'data/train/{categories}'):
        train_labels_not_categorical.append(number)
        img = cv2.imread(os.path.join(f'data/train/{categories}', filename))
        train_data.append(img)

train_labels = tf.keras.utils.to_categorical(train_labels)